# Compass.UOL - Sprint 4 & 5
## Equipe 3 - José Pedro, Pedro Montenegro, Natália Cardoso, Renan Mazzilli


## Dataset: [hotel_reservations](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset)

# Processamento de Dados e Análise Exploratória de Dados

### Imports e instalações necessárias

In [10]:
# Instalar as bibliotecas necessárias para o projeto
%pip install pandas boto3 joblib scikit-learn sagemaker seaborn

# Importar as bibliotecas necessárias 
import json
import boto3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

Note: you may need to restart the kernel to use updated packages.


### Configurações gerais do projeto e variáveis globais

In [11]:
# Carregar configurações do arquivo JSON
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Atribuir as variáveis de configuração
bucket_name = config['bucket_name']
table_name = config['table_name']
model_metrics_table = config['model_metrics_table']
role_name = config['role_name']
dataset_path = config['dataset_path']
processed_dataset_path = config['processed_dataset_path']
model_path = config['model_path']
profile_name = config['profile_name']
aws_region = config['aws_region']

# Inicialização da sessão AWS
session = boto3.Session(profile_name=profile_name, region_name=aws_region)
s3_client = session.client('s3')
dynamodb_client = session.client('dynamodb')
iam_client = session.client('iam')

### Pré-processamento dos dados (verificação de valores nulos, tratamento de variáveis categóricas)

Com relação ao pré-processamento dos dados, foram realizadas as seguintes etapas:

- **Normalização e Padrionização:** etapa que visa normalizar e padronizar os dados, a fim de evitar que variáveis com escalas diferentes possam influenciar o modelo de forma desproporcional. Para isso, utilizamos a função `StandardScaler` do pacote `sklearn.preprocessing`.

*Kuhn, M., & Johnson, K. (2013). Applied Predictive Modeling. Springer. Chapter on Data Preprocessing.*

- **Tratamento de Valores Ausentes:** etapa que visa tratar valores ausentes, que podem prejudicar o modelo. 

*Little, R. J., & Rubin, D. B. (2019). Statistical Analysis with Missing Data. Wiley. Missing Data Handling Techniques.*

- **Remoção de Outliers:** etapa que visa remover outliers, que podem prejudicar o modelo devido a sua influência desproporcional.

*Aggarwal, C. C. (2015). Outlier Analysis. Springer. Techniques for Outlier Detection and Treatment.*

- **Feature Engineering:** etapa que visa criar novas variáveis a partir das variáveis originais, a fim de melhorar o desempenho do modelo.

*Zheng, A., & Casari, A. (2018). Feature Engineering for Machine Learning: Principles and Techniques for Data Scientists. O'Reilly Media. Importance of Feature Engineering.*


In [ ]:
# Pré-processamento de Dados
class DataPreprocessor:
    def __init__(self, dataset_path, processed_dataset_path):
        self.dataset_path = dataset_path
        self.processed_dataset_path = processed_dataset_path

    def load_data(self):
        self.dataset = pd.read_csv(self.dataset_path)

    def preprocess_data(self):
        # Convertendo as colunas categóricas em numéricas
        self.dataset = pd.get_dummies(self.dataset, columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'booking_status'])

        # Deixando o Booking_ID com valores numéricos (removendo o INN)
        self.dataset['Booking_ID'] = self.dataset['Booking_ID'].str.extract('(\d+)', expand=False).astype(float)

        # Criar a nova coluna label_avg_price_per_room
        self.dataset['label_avg_price_per_room'] = pd.cut(self.dataset['avg_price_per_room'], bins=[0, 85, 115, float('inf')], labels=[1, 2, 3])

        # Excluir a coluna avg_price_per_room
        self.dataset.drop(columns=['avg_price_per_room'], inplace=True)

        # Remover linhas com valores ausentes
        self.dataset.dropna(inplace=True)

        # Remoção de outliers
        numerical_cols = self.dataset.select_dtypes(include=['float64', 'int64']).columns

        # Calcular os quantis e IQR
        Q1 = self.dataset[numerical_cols].quantile(0.25)
        Q3 = self.dataset[numerical_cols].quantile(0.75)
        IQR = Q3 - Q1

        # Filtrar outliers baseando-se no IQR
        self.dataset = self.dataset[~((self.dataset[numerical_cols] < (Q1 - 1.5 * IQR)) | (self.dataset[numerical_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

        # Normalização/Padrionização das features numéricas
        scaler = StandardScaler()
        self.dataset[numerical_cols] = scaler.fit_transform(self.dataset[numerical_cols])

        # Criação de novas features (feature engineering)
        self.dataset['total_guests'] = self.dataset['no_of_adults'] + self.dataset['no_of_children'] + self.dataset['no_of_weekend_nights'] + self.dataset['no_of_week_nights']
        self.dataset['total_nights'] = self.dataset['no_of_weekend_nights'] + self.dataset['no_of_week_nights']

        # Salvando o dataset preprocessado
        self.dataset.to_csv(self.processed_dataset_path, index=False)

# Executar o pré-processamento de dados
preprocessor = DataPreprocessor(dataset_path, processed_dataset_path)
preprocessor.load_data()
preprocessor.preprocess_data()

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\peter\AppData\Local\Temp\ipykernel_8200\3888047351.py:17: SyntaxWarning: invalid escape sequence '\d'
  self.dataset['Booking_ID'] = self.dataset['Booking_ID'].str.extract('(\d+)', expand=False).astype(float)


Credenciais configuradas corretamente. Buckets disponíveis:
  hotel-reservations-bucket-test
  hotel-reservations-bucket-test2
  hotel-reservations-bucket-test3
  modelbuckettest-compass-sprint4-5
  sagemaker-studio-381491839981-1ckrmnc4h8i
  sagemaker-studio-381491839981-46ozrms19x4
  sagemaker-studio-381491839981-6fc7cnmbqei
  sagemaker-studio-381491839981-7lmdzvgunt3
  sagemaker-studio-381491839981-bbjioi54pya
  sagemaker-studio-381491839981-gyom2iijcpa
  sagemaker-studio-381491839981-jadoe9fgwu
  sagemaker-studio-381491839981-p3c2dxmgle
  sagemaker-us-east-1-381491839981
  testing-sagemaker-train


### Upload do dataset para o AWS DynamoDB na tabela configurada

In [ ]:
# Classe para upload do Dataset pré-processado para o DynamoDB
class DynamoDBUploader:
    # Inicialização da classe com os parâmetros necessários
    def __init__(self, processed_dataset_path, table_name, profile_name, aws_region):
        self.processed_dataset_path = processed_dataset_path
        self.table_name = table_name
        self.profile_name = profile_name
        self.aws_region = aws_region
        self.dataset = pd.read_csv(self.processed_dataset_path)
    # Função para fazer o upload dos dados para o DynamoDB
    def upload_to_dynamodb(self):
        session = boto3.Session(profile_name=self.profile_name, region_name=self.aws_region)
        dynamodb = session.resource('dynamodb')
        table = dynamodb.Table(self.table_name)
        with table.batch_writer() as batch:
            for index, row in self.dataset.iterrows():
                item = {
                    'Booking_ID': str(row['Booking_ID'])
                }
                for key, value in row.items():
                    if key != 'Booking_ID':
                        item[key] = value if not pd.isna(value) else None
                batch.put_item(Item=item)

    # Verificar as credenciais AWS configuradas verificando se é possível listar os buckets S3
    def check_aws_credentials(self):
        try:
            session = boto3.Session(profile_name=self.profile_name)
            s3 = session.client('s3')
            response = s3.list_buckets()
            print('Credenciais configuradas corretamente. Buckets disponíveis:')
            for bucket in response['Buckets']:
                print(f'  {bucket["Name"]}')
        except NoCredentialsError:
            print('Credenciais não encontradas. Configure suas credenciais AWS.')
        except PartialCredentialsError:
            print('Credenciais incompletas. Verifique suas credenciais AWS.')
        except Exception as e:
            print(f'Erro ao verificar as credenciais: {e}')


# Chamada dos métodos da classe para realizar o upload dos dados para o DynamoDB
uploader = DynamoDBUploader(processed_dataset_path, table_name, profile_name, aws_region)
uploader.check_aws_credentials()
uploader.upload_to_dynamodb()